# Daily Log to SQL

This is a file to help clean up data from the daily logs and insert them into the Limblab MySQL database. You will need to know the sesames and either be connected to the VPN or running this remotely on Shrek or Donkey to use this.


## Required Dependencies:

- sqlalchemy
- pymysql
- numpy
- pandas


### Linux specific
You'll need to run <code> sudo apt install libmysqlclient mysql-client-core </code>

### macOS specific
You'll need to run <code> brew install mysql </code>

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from os import path, system
from sys import platform
import glob

### Background data

what is the monkey's name? What directory are we going to be scraping (better to do this once per monkey)

In [2]:
# Monkey data
monkeyName = "Greyson"
ccmID = "18E2"

# directory info
scrape_dir = '/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData'
if not path.exists(scrape_dir):
    print(f"{scrape_dir} does not exist! Try again!")

### Get a list of all threshold files

We'll work with nev and plx files for now. Going to assume that all recordings have those, so we'll work off that

In [3]:
# # List of files. Going to take a minute
# if 'win' in platform:
#     _ = input()
#     file_list = system(f"dir /B /S {scrape_dir}\\*.nev ")
#     print(file_list[-1])
# if 'nix' in platform:
#     file_list = system(f"ls -R {scrape_dir}/*.nev")

nev_list = glob.glob(f"{scrape_dir}/*/*.nev")

In [4]:
nev_list

['/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData/20181220/20181220_Greyson_FreeReaching_001.nev',
 '/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData/20181220/20181220_Greyson_PG_001.nev',
 '/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData/20181205/20181205_Greyson_PG_002.nev',
 '/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData/20181205/20181205_Greyson_PG_001.nev',
 '/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData/20200213/20200213_Greyson_Cage_003003.nev',
 '/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData/20200213/20200213_Greyson_Cage_005005.nev',
 '/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData/20200213/20200213_Greyson_Cage_010010.nev',
 '/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData/20200213/20200213_Greyson_Cage_004004.nev',
 '/home/kevin/Documents/L_MillerLab/data/Greyson_17L2/CerebusData/20200213/20200213_Greyson_Cage_016016.nev',
 '/home/kevin/Docume

In [ ]:
file_list